Assignment 3(SVM only vs PCA+SVM)

Author: Xiaodi Sun

Uni:xs2315

In [12]:
# firstly we import some necessary packages here
import numpy as np
import scipy.io as spio
import sklearn.svm as svm
import sklearn.model_selection as ms
from sklearn.decomposition import PCA
import os

import multiprocessing as mp

from time import sleep
import timeit


In [13]:
# Now we do some data preperation

class Trail(object):
        
    """
        
    This class is used for recording the trail and its accuracy
        
    """
    

    
    def __init__(self,test_id,withPCA,components,gamma,brainmask,random_state,c_value,accuracy,real_labels,predicted_labels):
        
       
        
        self.id = test_id


        self.withPCA = withPCA
        
        self.num_of_componnets = components
        self.gamma = gamma
        self.brainmask = brainmask
        self.random_state = random_state
        self.c_value = c_value
        self.accuracy = accuracy
        self.is_max_accuracy = False
        self.real_labels = real_labels
        self.predicted_labels = predicted_labels
        
        
        
    def __str__(self):
        
        if self.is_max_accuracy:
            
            if self.withPCA:
            
                return  "(Best)Trail {0:d} : PCA(components = {1:d}), SVM(C = {2:.1f}), gamma = {3:f}), random state = {4:d} , brainmask(Threshold = {5:d}) Accuracy = {6:.4f} % ".format(self.id,self.\
                                                  num_of_componnets,self.c_value,self.gamma,\
                                      self.random_state,self.brainmask,self.accuracy*100)+ \
                    "For the best trail (trail {0:d}), Real Labels:".format(self.id) +\
                    str(self.real_labels) + ", Predicted Labels: "+ str(self.predicted_labels)+"\n"

            else:
                
                
                 return  "(Best)Trail {0:d} :SVM(C = {2:.1f}), gamma = {3:f}), random state = {4:d}, brainmask(Threshold = {5:d}) Accuracy = {6:.4f} % ".format(self.id,self.\
                                              num_of_componnets,self.c_value,self.gamma,\
                                  self.random_state,self.brainmask,self.accuracy*100)+ \
                "For the best trail (trail {0:d}), Real Labels:".format(self.id) +\
                str(self.real_labels) + ", Predicted Labels: "+ str(self.predicted_labels)+"\n"

        else:
            
            if self.withPCA:
                
        
                return "Trail {0:d} : PCA(components = {1:d}), SVM(C = {2:.1f}), gamma = {3:f}), random state = {4:d} , brainmask(Threshold = {5:d})  Accuracy = {6:.4f} % \n".format(self.id,self.\
                                                  num_of_componnets,self.c_value,self.gamma,\
                                      self.random_state,self.brainmask,self.accuracy*100)
            else:
                return "Trail {0:d} : SVM(C = {2:.1f}), gamma = {3:f}) , random state = {4:d} , brainmask(Threshold = {5:d})  Accuracy = {6:.4f} % \n".format(self.id,self.\
                                              num_of_componnets,self.c_value,self.gamma,\
                                  self.random_state,self.brainmask,self.accuracy*100)
    
    
    def __repr__(self):
        return self.__str__()
    
    
    
    def is_max (self):
        self.is_max_accuracy = True
        
    def get_accuracy(self):
        return self.accuracy
    
    
    
    
    
    
    
        

In [14]:
#loading 

label_test = spio.loadmat('label.mat',squeeze_me = True)
label_retest = spio.loadmat('label.mat',squeeze_me = True)
data = {}
data_retest ={}

data[200]  = spio.loadmat('sub-01/ses-test/func/fMRImask_200.mat')
data[300]  = spio.loadmat('sub-01/ses-test/func/fMRImask_300.mat')
data[400]  = spio.loadmat('sub-01/ses-test/func/fMRImask_400.mat')
data[500] = spio.loadmat('sub-01/ses-test/func/fMRImask.mat')
data[600]  = spio.loadmat('sub-01/ses-test/func/fMRImask_600.mat')

data_retest[200]  = spio.loadmat('sub-01/ses-retest/func/fMRImask_200.mat')
data_retest[300]  = spio.loadmat('sub-01/ses-retest/func/fMRImask_300.mat')
data_retest[400]  = spio.loadmat('sub-01/ses-retest/func/fMRImask_400.mat')
data_retest[500] = spio.loadmat('sub-01/ses-retest/func/fMRImask.mat')
data_retest[600]  = spio.loadmat('sub-01/ses-retest/func/fMRImask_600.mat')

# ses-test

In [15]:




def test(rs,i,j,k,c,test_id,withPCA,conn):
    
    X_train, X_test, Y_train, Y_test = ms.train_test_split(data[i]['fMRIdata_1D_ROI'],
            label_test['label'],test_size = 0.5,random_state=rs)
        

    
    if withPCA:
        
        
        
        
        pca = PCA(n_components=j)
        pca.fit(X_train)

        X_pca_train = pca.transform(X_train)
        X_pca_test = pca.transform(X_test)

        model = svm.SVC(C=c, cache_size=400, class_weight=None, coef0=1.0,
            decision_function_shape='ovr', degree=114, gamma=k, kernel='linear',
            max_iter=-1, probability=False, random_state=rs, shrinking=False,
                        tol=0.01, verbose=False)
        model.fit(X_pca_train,Y_train)

        predicted = model.predict(X_pca_test)
        score = model.score(X_pca_test,Y_test)


        conn.send(Trail(test_id,withPCA,j,k,i,rs,c,score,Y_test,predicted))


    else:
        
        model = svm.SVC(C=c, cache_size=400, class_weight=None, coef0=1.0,
            decision_function_shape='ovr', degree=114, gamma=k, kernel='linear',
            max_iter=-1, probability=False, random_state=None, shrinking=False,
                        tol=0.01, verbose=False)
        model.fit(X_train,Y_train)

        predicted = model.predict(X_test)
        score = model.score(X_test,Y_test)


        conn.send(Trail(test_id,withPCA,j,k,i,rs,c,score,Y_test,predicted))
        
    conn.close()
        
        

In [16]:
log_list_1 = []
log_list_2 = []
test_id_1 = 0
test_id_2 = 0
start = timeit.default_timer()
for i in [200,300,400,500,600]:
    for rs in [41,42]:
   
        
        processes = []
        parent_conn_list_1 = []
        parent_conn_list_2 = []

        for j in [50,70,90,100]:

            for k in [0.01,0.1]:

                for c in [90.0,100.0]:
                    
                    parent_conn , child_conn = mp.Pipe(False)
                    

                    test_id_1 = test_id_1 + 1
                    test_id_2 = test_id_2 + 1

                    p_1 = mp.Process(target = test,args=(rs,i,j,k,c,test_id_1,True,child_conn))
                    processes.append(p_1)
                    p_1.start()
                    parent_conn_list_1.append(parent_conn)
                    parent_conn_f , child_conn_f = mp.Pipe(False)
                    
                    p_2 =mp.Process(target = test,args=(rs,i,j,k,c,test_id_2,False,child_conn_f))
                    processes.append(p_2)
                    p_2.start()
                    parent_conn_list_2.append(parent_conn_f)
                    
               
                
        for p in processes:

            p.join()

        for r in parent_conn_list_1:

            log_list_1.append(r.recv())
            r.close()
        
        for r in parent_conn_list_2:

            log_list_2.append(r.recv())
            r.close()


                    


          
   

In [17]:
max_accuracy = 0

for i in log_list_1:
    if i.get_accuracy() > max_accuracy:
        max_accuracy = i.get_accuracy()
                    
    
                    
for i in log_list_1:
    if i.get_accuracy() == max_accuracy:
        i.is_max()
        
if os.path.exists("log_1.txt"):
    os.remove("log_1.txt")
    
f = open("log_1.txt", "w")
for trail in log_list_1:
    
    if not trail.is_max_accuracy:
        
        print(trail)
   
        print (trail, file = f)

for trail in log_list_1:
    
    if  trail.is_max_accuracy:
        
        print(trail)
   
        print (trail, file = f)

f.close()
      
    


Trail 1 : PCA(components = 50), SVM(C = 90.0), gamma = 0.010000), random state = 41 , brainmask(Threshold = 200)  Accuracy = 75.0000 % 

Trail 2 : PCA(components = 50), SVM(C = 100.0), gamma = 0.010000), random state = 41 , brainmask(Threshold = 200)  Accuracy = 75.0000 % 

Trail 3 : PCA(components = 50), SVM(C = 90.0), gamma = 0.100000), random state = 41 , brainmask(Threshold = 200)  Accuracy = 75.0000 % 

Trail 4 : PCA(components = 50), SVM(C = 100.0), gamma = 0.100000), random state = 41 , brainmask(Threshold = 200)  Accuracy = 75.0000 % 

Trail 5 : PCA(components = 70), SVM(C = 90.0), gamma = 0.010000), random state = 41 , brainmask(Threshold = 200)  Accuracy = 78.2609 % 

Trail 6 : PCA(components = 70), SVM(C = 100.0), gamma = 0.010000), random state = 41 , brainmask(Threshold = 200)  Accuracy = 78.2609 % 

Trail 7 : PCA(components = 70), SVM(C = 90.0), gamma = 0.100000), random state = 41 , brainmask(Threshold = 200)  Accuracy = 78.2609 % 

Trail 8 : PCA(components = 70), SVM(C 

In [18]:
max_accuracy = 0

for i in log_list_2:
    if i.get_accuracy() > max_accuracy:
        max_accuracy = i.get_accuracy()
                    
    
                    
for i in log_list_2:
    if i.get_accuracy() == max_accuracy:
        i.is_max()
        
if os.path.exists("log_2.txt"):
    os.remove("log_2.txt")
    
f = open("log_2.txt", "w")
for trail in log_list_2:
    
    if not trail.is_max_accuracy:
        
        print(trail)
   
        print (trail, file = f)

for trail in log_list_2:
    
    if  trail.is_max_accuracy:
        
        print(trail)
   
        print (trail, file = f)

f.close()
      
    
stop = timeit.default_timer()
print (stop - start) 

Trail 1 : SVM(C = 90.0), gamma = 0.010000) , random state = 41 , brainmask(Threshold = 200)  Accuracy = 81.5217 % 

Trail 2 : SVM(C = 100.0), gamma = 0.010000) , random state = 41 , brainmask(Threshold = 200)  Accuracy = 81.5217 % 

Trail 3 : SVM(C = 90.0), gamma = 0.100000) , random state = 41 , brainmask(Threshold = 200)  Accuracy = 81.5217 % 

Trail 4 : SVM(C = 100.0), gamma = 0.100000) , random state = 41 , brainmask(Threshold = 200)  Accuracy = 81.5217 % 

Trail 5 : SVM(C = 90.0), gamma = 0.010000) , random state = 41 , brainmask(Threshold = 200)  Accuracy = 81.5217 % 

Trail 6 : SVM(C = 100.0), gamma = 0.010000) , random state = 41 , brainmask(Threshold = 200)  Accuracy = 81.5217 % 

Trail 7 : SVM(C = 90.0), gamma = 0.100000) , random state = 41 , brainmask(Threshold = 200)  Accuracy = 81.5217 % 

Trail 8 : SVM(C = 100.0), gamma = 0.100000) , random state = 41 , brainmask(Threshold = 200)  Accuracy = 81.5217 % 

Trail 9 : SVM(C = 90.0), gamma = 0.010000) , random state = 41 , bra

# ses-retest

In [19]:
log_list_retest_1 = []

log_list_retest_2 = []


max_accuracy_retest = 0.0

    
def retest(rs,i,j,k,c,test_id,withPCA,conn):
    
    X_train, X_test, Y_train, Y_test = ms.train_test_split(data_retest[i]['fMRIdata_1D_ROI'],
            label_test['label'],test_size = 0.5,random_state=rs)
        

    
    if withPCA:
        
        
        
        
        pca = PCA(n_components=j)
        pca.fit(X_train)

        X_pca_train = pca.transform(X_train)
        X_pca_test = pca.transform(X_test)

        model = svm.SVC(C=c, cache_size=800, class_weight=None, coef0=1.0,
            decision_function_shape='ovr', degree=14, gamma=k, kernel='linear',
            max_iter=-1, probability=False, random_state=rs, shrinking=False,
                        tol=0.01, verbose=False)
        model.fit(X_pca_train,Y_train)

        predicted = model.predict(X_pca_test)
        score = model.score(X_pca_test,Y_test)


        conn.send(Trail(test_id,withPCA,j,k,i,rs,c,score,Y_test,predicted))
#     conn.send([test_id,score])

    else:
        
        model = svm.SVC(C=c, cache_size=800, class_weight=None, coef0=1.0,
            decision_function_shape='ovr', degree=14, gamma=k, kernel='linear',
            max_iter=-1, probability=False, random_state=rs, shrinking=False,
                        tol=0.01, verbose=False)
        model.fit(X_train,Y_train)

        predicted = model.predict(X_test)
        score = model.score(X_test,Y_test)


        conn.send(Trail(test_id,withPCA,j,k,i,rs,c,score,Y_test,predicted))
        
    conn.close()
        
    


In [20]:
test_id_1 = 0
test_id_2 = 0



start = timeit.default_timer()
for i in [200,300,400,500,600]:
    for rs in [41,42]:
        processes = []
        parent_conn_list_1 = []
        parent_conn_list_2 = []


        for j in [70,80,90,100]:

            for k in [0.01,0.1]:

                for c in [90.0,100.0,110.0]:
                    
                    parent_conn , child_conn = mp.Pipe(False)
                    
#                     processes.append(mp.Process(target = retest,args=(rs,i,j,k,c,child_conn)))
                    test_id_1 = test_id_1 + 1
                    test_id_2 = test_id_2 + 1
#                     processes.append(mp.Process(target = retest,args=(rs,i,j,k,c,test_id,True,child_conn)))
                    p_1 = mp.Process(target = retest,args=(rs,i,j,k,c,test_id_1,True,child_conn))
                    processes.append(p_1)
                    p_1.start()
                    parent_conn_list_1.append(parent_conn)
                    parent_conn_f , child_conn_f = mp.Pipe(False)
             
                    p_2 =mp.Process(target = retest,args=(rs,i,j,k,c,test_id_2,False,child_conn_f))
                    processes.append(p_2)
                    p_2.start()
                    parent_conn_list_2.append(parent_conn_f)
                    
               
                
        for p in processes:

            p.join()
            
        for r in parent_conn_list_1:

            log_list_retest_1.append(r.recv())
            r.close()
        
        for r in parent_conn_list_2:

            log_list_retest_2.append(r.recv())
            r.close()
                
                
                

In [21]:
for i in log_list_retest_1:
    if i.get_accuracy() > max_accuracy_retest:
        max_accuracy_retest = i.get_accuracy()
                    
                    
for i in log_list_retest_1:
    if i.get_accuracy() == max_accuracy_retest:
        i.is_max()
        
if os.path.exists("log_retest_1.txt"):
    os.remove("log_retest_1.txt")
    
    
f = open("log_retest_1.txt", "w")

for trail in log_list_retest_1:
    
    if not trail.is_max_accuracy:
        
        print (trail, file = f)
        print(trail)
        

for trail in log_list_retest_1:
    
    if  trail.is_max_accuracy:
        
        print (trail, file = f)
        print(trail)

f.close()



Trail 1 : PCA(components = 70), SVM(C = 90.0), gamma = 0.010000), random state = 41 , brainmask(Threshold = 200)  Accuracy = 71.7391 % 

Trail 2 : PCA(components = 70), SVM(C = 100.0), gamma = 0.010000), random state = 41 , brainmask(Threshold = 200)  Accuracy = 71.7391 % 

Trail 3 : PCA(components = 70), SVM(C = 110.0), gamma = 0.010000), random state = 41 , brainmask(Threshold = 200)  Accuracy = 71.7391 % 

Trail 4 : PCA(components = 70), SVM(C = 90.0), gamma = 0.100000), random state = 41 , brainmask(Threshold = 200)  Accuracy = 71.7391 % 

Trail 5 : PCA(components = 70), SVM(C = 100.0), gamma = 0.100000), random state = 41 , brainmask(Threshold = 200)  Accuracy = 71.7391 % 

Trail 6 : PCA(components = 70), SVM(C = 110.0), gamma = 0.100000), random state = 41 , brainmask(Threshold = 200)  Accuracy = 71.7391 % 

Trail 7 : PCA(components = 80), SVM(C = 90.0), gamma = 0.010000), random state = 41 , brainmask(Threshold = 200)  Accuracy = 72.8261 % 

Trail 8 : PCA(components = 80), SVM(C